In [1]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from sklearn.metrics import (confusion_matrix, precision_score, auc,
                             roc_curve, recall_score, classification_report, f1_score)
#import pickle
#import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
#import seaborn as sns
#from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import scipy.stats as stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
from sklearn.cluster import KMeans
#from sklearn_som.som import SOM
from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
import glob
#import sys
#np.set_printoptions(threshold=sys.maxsize)

2022-12-09 18:02:15.801915: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 18:02:15.801961: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Dados NSL-KDD

In [2]:
#Carrega dados do dataset
#monday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Monday-WorkingHours.csv", skipinitialspace = True)
#tuesday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Tuesday-WorkingHours.csv", skipinitialspace = True)
#wednesday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Wednesday-WorkingHours.csv", skipinitialspace = True)
#thursday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Thursday-WorkingHours.csv", skipinitialspace = True)
#friday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Friday-WorkingHours.csv", skipinitialspace = True)

# Carregamento Datasets

In [3]:
#Carrega dados do dataset
monday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Monday-WorkingHours.csv", skipinitialspace = True)
tuesday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Tuesday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((monday, tuesday), axis=0)
del monday, tuesday

In [4]:
wednesday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Wednesday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((x, wednesday), axis=0)
del wednesday

In [5]:
thursday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Thursday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((x, thursday), axis=0)
del thursday

In [6]:
friday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Friday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((x, friday), axis=0)
del friday

In [7]:
#Concatena todos os datasets carregados

#x = pd.concat((monday, friday1, friday2, friday3), axis=0)
#x = pd.concat((monday, tuesday, wednesday, thursday, friday), axis=0)
x = x.drop(['Flow ID', 'Src IP', 'Dst IP', 'Src Port', 'Dst Port', 'Timestamp'], axis = 1)

In [8]:
#Mostra todas as classes possíveis para a label
#x['Label'].unique()

In [9]:
#Transforma o dataset multiclasse para binário
x['Label'] = x['Label'].replace('BENIGN', 0)
x['Label'] = x['Label'].replace('FTP-Patator', 1)
x['Label'] = x['Label'].replace('SSH-Patator', 1)
x['Label'] = x['Label'].replace('FTP-Patator - Attempted', 0)
x['Label'] = x['Label'].replace('SSH-Patator - Attempted', 0)
x['Label'] = x['Label'].replace('DoS slowloris', 1)
x['Label'] = x['Label'].replace('DoS slowloris - Attempted', 0)
x['Label'] = x['Label'].replace('DoS Slowhttptest', 1)
x['Label'] = x['Label'].replace('DoS Slowhttptest - Attempted', 0)
x['Label'] = x['Label'].replace('DoS Hulk', 1)
x['Label'] = x['Label'].replace('DoS Hulk - Attempted', 0)
x['Label'] = x['Label'].replace('DoS GoldenEye', 1)
x['Label'] = x['Label'].replace('Heartbleed', 1)
x['Label'] = x['Label'].replace('DoS GoldenEye - Attempted', 0)
x['Label'] = x['Label'].replace('Web Attack - Brute Force', 1)
x['Label'] = x['Label'].replace('Web Attack - Brute Force - Attempted', 0)
x['Label'] = x['Label'].replace('Web Attack - XSS', 1)
x['Label'] = x['Label'].replace('Web Attack - XSS - Attempted', 0)
x['Label'] = x['Label'].replace('Web Attack - Sql Injection', 1)
x['Label'] = x['Label'].replace('Infiltration', 1)
x['Label'] = x['Label'].replace('Infiltration - Attempted', 0)
x['Label'] = x['Label'].replace('Bot', 1)
x['Label'] = x['Label'].replace('Bot - Attempted', 0)
x['Label'] = x['Label'].replace('DDoS', 1)
x['Label'] = x['Label'].replace('PortScan', 1)

In [10]:
x.replace([np.inf, -np.inf], np.nan, inplace=True)
x.dropna(inplace=True)

In [11]:
#Realiza o One-Hot Encoding
#x = pd.get_dummies(x)

In [12]:
#Separa as labels e os Dados em datasets diferentes
data_class = x['Label']
data = x.drop(['Label'], axis = 1) 

In [13]:
data = data.to_numpy()
#transforma dados infinitos em finitos
#data = np.nan_to_num(data,posinf=10000000, neginf=10000000)
data_class = data_class.to_numpy()

In [14]:
noTrain = np.count_nonzero(data_class == 0)
anTrain = np.count_nonzero(data_class == 1)

In [15]:
print("Dados Normais: ", noTrain, "\nDados Anômalos: ", anTrain)

Dados Normais:  1666172 
Dados Anômalos:  433849


# Abordagem Autoencoder-Algoritmos

In [16]:
epochs = 50
batch_size = 256
optimizer = Adam(learning_rate=0.0001)
activation = "relu"
neighbors = 11

num_folds = 10

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

fold_no = 1
count = 0

#Métricas ErroRecons
matrix_ErroRecons = np.zeros((num_folds,2,2))
accuracy_ErroRecons = np.zeros(num_folds)
recall_ErroRecons = np.zeros(num_folds)
precision_ErroRecons = np.zeros(num_folds)
f1_ErroRecons = np.zeros(num_folds)

#Métricas KNN
matrix_knn = np.zeros((num_folds,2,2))
accuracy_knn = np.zeros(num_folds)
recall_knn = np.zeros(num_folds)
precision_knn = np.zeros(num_folds)
f1_knn = np.zeros(num_folds)

#Métricas Decision Tree
matrix_dt = np.zeros((num_folds,2,2))
accuracy_dt = np.zeros(num_folds)
recall_dt = np.zeros(num_folds)
precision_dt = np.zeros(num_folds)
f1_dt = np.zeros(num_folds)

#Métricas SVM
matrix_svm = np.zeros((num_folds,2,2))
accuracy_svm = np.zeros(num_folds)
recall_svm = np.zeros(num_folds)
precision_svm = np.zeros(num_folds)
f1_svm = np.zeros(num_folds)


for train, test in kfold.split(data, data_class):
    
    #####################################################
    ####Pré-Processamento
    
    #Separação dos Folds de Treinamento e Teste
    dataTrain = data[train]
    dataTrain_class = data_class[train]
    modelTest = data[test]
    modelTest_class = data_class[test]
    
    #Conta Quantidade de dados
    Train = dataTrain.shape[1]
    noTrain = np.count_nonzero(dataTrain_class == 0)
    anTrain = np.count_nonzero(dataTrain_class == 1)

    #Separa dados normais e anômalos
    normalTrain = dataTrain[np.where(dataTrain_class == 0)]
    anomalyTrain = dataTrain[np.where(dataTrain_class == 1)]
    
    #Divide dados para treinamento
    porcen = 1
    j = int(anTrain * porcen)
    
    #Índices escolhidos aleatoriamente
    numbers_knn_normal = np.array(random.sample(range(0, noTrain),j))
    numbers_knn_anomaly = np.array(random.sample(range(0,anTrain),j))
    numbers_autoencoder_normal = np.array(list(set(np.arange(0, noTrain)) - set(numbers_knn_normal)))
    
    #Dados KNN
    knnTrain_normal = normalTrain[numbers_knn_normal]
    knnTrain_anomaly = anomalyTrain[numbers_knn_anomaly]
    knnTrain = np.concatenate((knnTrain_normal, knnTrain_anomaly), axis=0)
    knnTrain_class = np.concatenate((np.zeros(j), np.ones(j)))
    
    #Dados Autoencoder
    autoencoderTrain = normalTrain[numbers_autoencoder_normal]
    autoencoderTrain_class = np.zeros(autoencoderTrain.shape[0])
    
    #Normalização
    scaler = MinMaxScaler()
    scaler.fit(knnTrain)
    autoencoderTrain = scaler.transform(autoencoderTrain)
    knnTrain = scaler.transform(knnTrain)
    modelTest = scaler.transform(modelTest)
    
    #####################################################
    ####Autoencoder
    #Modela as camadas do Autoencoder
    var = autoencoderTrain.shape[1]
    input_vector = keras.Input(shape=(var,))
    en1 = layers.Dense(int(var/2), activation=activation)(input_vector)
    en2 = layers.Dense(int(var/4), activation=activation)(en1)
    de1 = layers.Dense(int(var/2), activation=activation)(en2)
    de2 = layers.Dense(var, activation=activation)(de1)

    #Gera o modelo
    autoencoder = keras.Model(input_vector, de2)
    
    #Gera modelo do Encoder
    encoder = keras.Model(input_vector, en2)
    
    #Cria Decoder
    encoded_input = keras.Input(shape=(int(var/4),))
    dec_layer = autoencoder.layers[-2](encoded_input)
    decoder_layer = autoencoder.layers[-1](dec_layer)
    decoder = keras.Model(encoded_input, decoder_layer)

    #Compila o modelo
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")

    #Treina o modelo
    history = autoencoder.fit(autoencoderTrain, autoencoderTrain, epochs = epochs, batch_size = batch_size, shuffle = True)
    
    
    #####################################################
    ####KNN
    ###Treinamento KNN
    pred_knn = autoencoder.predict(knnTrain)
    mse_knn = np.mean(np.power(knnTrain - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                 'true_class': knnTrain_class})
    error_knn = error_df_knn.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_knn[:,0].reshape(-1,1), axis=1)
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(error_encoder, error_knn[:,1])
    
    #Teste do Modelo
    pred_knn = autoencoder.predict(modelTest)
    mse_knn = np.mean(np.power(modelTest - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                'true_class': modelTest_class})
    error_knn = error_df_knn.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer, error_knn[:,0].reshape(-1,1), axis=1)
    predict_knn = neigh.predict(error_encoder)
    
    #Métricas
    accuracy_knn[count] = accuracy_score(error_knn[:,1], predict_knn[:])
    recall_knn[count] = recall_score(error_knn[:,1], predict_knn[:])
    precision_knn[count] = precision_score(error_knn[:,1], predict_knn[:])
    f1_knn[count] = f1_score(error_knn[:,1], predict_knn[:])
    matrix_knn[count] = confusion_matrix(error_knn[:,1], predict_knn[:])
     
    #####################################################
    ####Decisition Tree
    ###Treinamento Decisition Tree 
    pred_dt = autoencoder.predict(knnTrain)
    mse_dt = np.mean(np.power(knnTrain - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': knnTrain_class})
    error_dt = error_df_dt.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_dt[:,0].reshape(-1,1), axis=1)
    dt = tree.DecisionTreeClassifier()
    dt.fit(error_encoder, error_dt[:,1])

    #Teste
    pred_dt = autoencoder.predict(modelTest)
    mse_dt = np.mean(np.power(modelTest - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': modelTest_class})
    error_dt = error_df_dt.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer,error_dt[:,0].reshape(-1,1), axis=1)
    predict_dt = dt.predict(error_encoder)

    #Métricas
    accuracy_dt[count] = accuracy_score(error_dt[:,1], predict_dt[:])
    recall_dt[count] = recall_score(error_dt[:,1], predict_dt[:])
    precision_dt[count] = precision_score(error_dt[:,1], predict_dt[:])
    f1_dt[count] = f1_score(error_dt[:,1], predict_dt[:])
    matrix_dt[count] = confusion_matrix(error_dt[:,1], predict_dt[:])
    
    #####################################################
    ####SVM
    #Treinamento SVM
    pred_svm = autoencoder.predict(knnTrain)
    mse_svm = np.mean(np.power(knnTrain - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                     'true_class': knnTrain_class})
    error_svm = error_df_svm.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_svm[:,0].reshape(-1,1), axis=1)
    clf = svm.SVC(kernel='rbf')
    clf.fit(error_encoder, error_svm[:,1])
    
    #Teste
    pred_svm = autoencoder.predict(modelTest)
    mse_svm = np.mean(np.power(modelTest - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                     'true_class': modelTest_class})
    error_svm = error_df_svm.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer,error_svm[:,0].reshape(-1,1), axis=1)
    predict_svm = clf.predict(error_encoder)
    
    #Métricas
    accuracy_svm[count] = accuracy_score(error_svm[:,1], predict_svm[:])
    recall_svm[count] = recall_score(error_svm[:,1], predict_svm[:])
    precision_svm[count] = precision_score(error_svm[:,1], predict_svm[:])
    f1_svm[count] = f1_score(error_svm[:,1], predict_svm[:])
    matrix_svm[count] = confusion_matrix(error_svm[:,1], predict_svm[:])
    
    
    Result_GridSearchLimiar = np.array(["KNN", "Acurácia", accuracy_knn[count], "",
                                        "Recall", recall_knn[count], "",
                                        "Precision", precision_knn[count], "",
                                        "F1", f1_knn[count], "-----------------","-----------------",
                                    "Decision Tree",  "Acurácia", accuracy_dt[count],"",
                                        "Recall", recall_dt[count], "",
                                        "Precision", precision_dt[count], "",
                                        "F1", f1_dt[count], "-----------------","-----------------",
                                    "SVM",  "Acurácia", accuracy_svm[count],"",
                                        "Recall", recall_svm[count], "",
                                        "Precision", precision_svm[count], "",
                                        "F1", f1_svm[count], "-----------------","-----------------","-----------------","-----------------",
                                    "KNN", matrix_knn[count], "-----------------","-----------------",
                                    "Decision Tree", matrix_dt[count], "-----------------","-----------------",
                                    "SVM", matrix_svm[count]])

    np.savetxt('Result_Parcial_CICIDS2017_IL+RE_Fold'+str(fold_no)+'.txt', Result_GridSearchLimiar, delimiter = ',',fmt='%s')
    
    count = count + 1
    fold_no = fold_no + 1

2022-12-09 18:03:10.492993: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 18:03:10.493155: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-09 18:03:10.493260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-09 18:03:10.493348: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-12-09 18:03:10.493427: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Epoch 1/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0023
Epoch 2/50
4333/4333 [==============================] - 10s 2ms/step - loss: 5.4809e-04
Epoch 3/50
4333/4333 [==============================] - 10s 2ms/step - loss: 5.2197e-04
Epoch 4/50
4333/4333 [==============================] - 10s 2ms/step - loss: 5.1287e-04
Epoch 5/50
4333/4333 [==============================] - 10s 2ms/step - loss: 5.0582e-04
Epoch 6/50
4333/4333 [==============================] - 10s 2ms/step - loss: 5.0256e-04
Epoch 7/50
4333/4333 [==============================] - 10s 2ms/step - loss: 5.0042e-04
Epoch 8/50
4333/4333 [==============================] - 10s 2ms/step - loss: 4.9887e-04
Epoch 9/50
4333/4333 [==============================] - 10s 2ms/step - loss: 4.9797e-04
Epoch 10/50
4333/4333 [==============================] - 10s 2ms/step - loss: 4.9725e-04
Epoch 11/50
4333/4333 [==============================] - 10s 2ms/step - loss: 4.9669e-04
Epoch 12/50
4333/4333 [===========

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:223: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0029
Epoch 2/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0024
Epoch 3/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0024
Epoch 4/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0024
Epoch 5/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0024
Epoch 6/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0024
Epoch 7/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0024
Epoch 8/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0024
Epoch 9/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0024
Epoch 10/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0023
Epoch 11/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0023
Epoch 12/50
4333/4333 [==============================] - 10s 2ms/step - lo

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:223: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


...

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:223: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0014
Epoch 2/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.8425e-04
Epoch 3/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.4479e-04
Epoch 4/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.3395e-04
Epoch 5/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.2414e-04
Epoch 6/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.1639e-04
Epoch 7/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.1423e-04
Epoch 8/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.1086e-04
Epoch 9/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.0922e-04
Epoch 10/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.0824e-04
Epoch 11/50
4333/4333 [==============================] - 11s 2ms/step - loss: 6.0750e-04
Epoch 12/50
4333/4333 [===========

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:223: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0022
Epoch 2/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0017
Epoch 3/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0017
Epoch 4/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0017
Epoch 5/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0017
Epoch 6/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0017
Epoch 7/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0017
Epoch 8/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0017
Epoch 9/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0017
Epoch 10/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0017
Epoch 11/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0017
Epoch 12/50
4333/4333 [==============================] - 10s 2ms/step - lo

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:223: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [17]:
print("Autoencoder - KNN:\n", np.mean(matrix_knn, axis=0), "\nAcurácia: ", np.mean(accuracy_knn), ' +/- ', np.std(accuracy_knn), "\nRecall: ", np.mean(recall_knn), ' +/- ', np.std(recall_knn), "\nPrecision: ", np.mean(precision_knn), ' +/- ', np.std(precision_knn), "\nF1: ", np.mean(f1_knn), ' +/- ', np.std(f1_knn))

print("\nAutoencoder - SVM:\n", np.mean(matrix_svm, axis=0), "\nAcurácia: ", np.mean(accuracy_svm), ' +/- ', np.std(accuracy_svm), "\nRecall: ", np.mean(recall_svm), ' +/- ', np.std(recall_svm), "\nPrecision: ", np.mean(precision_svm), ' +/- ', np.std(precision_svm), "\nF1: ", np.mean(f1_svm), ' +/- ', np.std(f1_svm))

print("\nAutoencoder - Decision Tree:\n", np.mean(matrix_dt, axis=0), "\nAcurácia: ", np.mean(accuracy_dt), ' +/- ', np.std(accuracy_dt), "\nRecall: ", np.mean(recall_dt), ' +/- ', np.std(recall_dt), "\nPrecision: ", np.mean(precision_dt), ' +/- ', np.std(precision_dt), "\nF1: ", np.mean(f1_dt), ' +/- ', np.std(f1_dt))

Autoencoder - KNN:
 [[1.651762e+05 1.441000e+03]
 [6.980000e+01 4.331510e+04]] 
Acurácia:  0.9928057864799855  +/-  0.00036287706540639227 
Recall:  0.9983911454018713  +/-  0.0011123019844150614 
Precision:  0.967812811086714  +/-  0.002572883826187759 
F1:  0.9828607859537752  +/-  0.0008342066473502489

Autoencoder - SVM:
 [[162545.8   4071.4]
 [   228.3  43156.6]] 
Acurácia:  0.9795254402338005  +/-  0.0059642563028491835 
Recall:  0.9947378011322023  +/-  0.0008672833477093055 
Precision:  0.9144316655235025  +/-  0.023906310090179263 
F1:  0.9527301696930411  +/-  0.013042350523638948

Autoencoder - Decision Tree:
 [[1.650217e+05 1.595500e+03]
 [2.640000e+01 4.335850e+04]] 
Acurácia:  0.9922767440173497  +/-  0.00015760293976272913 
Recall:  0.9993914937468032  +/-  0.0001406570338883854 
Precision:  0.9645087480561466  +/-  0.0007321126686457925 
F1:  0.9816401679627977  +/-  0.0003675567969558494


In [18]:
Result = np.array(["KNN", "Acurácia", accuracy_knn, np.mean(accuracy_knn), np.std(accuracy_knn),"",
                                "Recall", recall_knn, np.mean(recall_knn), np.std(recall_knn), "",
                                "Precision", precision_knn, np.mean(precision_knn), np.std(precision_knn), "",
                                "F1", f1_knn, np.mean(f1_knn), np.std(f1_knn), "-----------------","-----------------",
                  "Decision Tree",  "Acurácia", accuracy_dt, np.mean(accuracy_dt), np.std(accuracy_dt),"",
                                "Recall", recall_dt, np.mean(recall_dt), np.std(recall_dt), "",
                                "Precision", precision_dt, np.mean(precision_dt), np.std(precision_dt), "",
                                "F1", f1_dt, np.mean(f1_dt), np.std(f1_dt), "-----------------","-----------------",
                  "SVM",  "Acurácia", accuracy_svm, np.mean(accuracy_svm), np.std(accuracy_svm),"",
                                "Recall", recall_svm, np.mean(recall_svm), np.std(recall_svm), "",
                                "Precision", precision_svm, np.mean(precision_svm), np.std(precision_svm), "",
                                "F1", f1_svm, np.mean(f1_svm), np.std(f1_svm), "","",])

np.savetxt('Result_IL+RE.txt', Result, delimiter = ',',fmt='%s')

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == "":


In [19]:
Result_Matrix = np.array(["KNN", matrix_knn, np.mean(matrix_knn, axis=0), np.std(matrix_knn, axis=0), "-----------------","-----------------",
                          "Decision Tree", matrix_dt, np.mean(matrix_dt, axis=0), np.std(matrix_dt, axis=0), "-----------------","-----------------",
                          "SVM", matrix_svm, np.mean(matrix_svm, axis=0), np.std(matrix_svm, axis=0)])

np.savetxt('Result_IL+RE_Matrix.txt', Result_Matrix, delimiter = ',',fmt='%s')

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
